---

# ✅ **Project Explanation (Baseline Model – Early Stage)**

This model is only meant as an **initial baseline**, to make sure the pipeline works and to get a rough idea of performance. It is *not yet the final or optimized solution*. If this helps you, feel free to **upvote** :)

---

# **1. Importing Libraries & Loading Data**

* The dataset is loaded from Kaggle (`train.csv` and `test.csv`) with `date_id` used as the index.
* Unnecessary columns that do not exist in both train and test are removed to keep the same feature structure.
* Missing values are checked to understand data quality.

---


In [ ]:
import warnings
warnings.filterwarnings("ignore") 


In [ ]:
import pandas as pd

df = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/train.csv',index_col="date_id")
df_test = pd.read_csv('/kaggle/input/hull-tactical-market-prediction/test.csv',index_col="date_id")


# **2. Handling Missing Values**

Because this is time-series data, we don’t simply drop rows with missing values.
Instead, we use a **three-step filling strategy**:

1. `ffill` – forward fill (use previous valid value)
2. `bfill` – backward fill (use next valid value)
3. Remaining missing values → filled with **median**

This keeps the time-series structure intact and avoids data leakage.

---

In [ ]:
df.isna().sum()[(df.nunique() > 1) & (df.isna().sum() > 0)]

In [ ]:
missing_in_test = set(df_test.columns) - set(df.columns)
print(missing_in_test)


In [ ]:
df = df.drop(columns=['market_forward_excess_returns', 'risk_free_rate'])
df_test = df_test.drop(columns=['lagged_forward_returns', 'is_scored', 'lagged_risk_free_rate', 'lagged_market_forward_excess_returns'])


In [ ]:
df.isna().sum()[(df.nunique() > 1) & (df.isna().sum() > 0)]


In [ ]:
df_ffill = df.fillna(method='ffill')
df_bfill = df_ffill.fillna(method='bfill')
df_filled = df_bfill.fillna(df_bfill.median())


---
# **3. Splitting Features and Target**

* The target variable we want to predict is: **`forward_returns`**
* All other columns become features (`X`).

```python
target = "forward_returns"
y = df_filled[target]
X = df_filled.drop(columns=[target])
```

---
# **4. Feature Scaling**

* Features are standardized using **StandardScaler**.
* The scaler is saved (`scaler.pkl`) so that the exact same scaling can be applied to the test set during prediction.
* This is required because LightGBM + Optuna will use these scaled values.

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

# ================= 0. PREPROCESS =================
# misal df_filled sudah siap
target = "forward_returns"
y = df_filled[target]
X = df_filled.drop(columns=[target])


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# simpan scaler
import joblib
joblib.dump(scaler, "scaler.pkl")


# **5. Hyperparameter Tuning with Optuna + TimeSeriesSplit**

This is the core idea of the baseline:

* We use **Optuna** to automatically search for the best LightGBM hyperparameters.
* **TimeSeriesSplit (5 folds)** is used for cross-validation to prevent data leakage in time-series.
* Each trial trains a model on past data and validates on future data.
* The average **RMSE (Root Mean Squared Error)** is returned as the objective metric.

If you find this useful — **upvote appreciated 🙌**

---

# **6. Training the Final Model**

After finding the best parameters, we:

* Retrain LightGBM on **the entire dataset** using those best parameters.
* This is still considered an **early-stage model**. It’s only for baseline purposes — not meant for final evaluation or competition submission.

---

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

tscv = TimeSeriesSplit(n_splits=5)
# ================= 1. DEFINE OBJECTIVE =================
def objective(trial):

    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 20, 256),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
    }

    rmse_scores = []

    # Cross-validation time series
    for train_index, valid_index in tscv.split(X):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        train_set = lgb.Dataset(X_train, y_train)
        valid_set = lgb.Dataset(X_valid, y_valid, reference=train_set)

        model = lgb.train(
            param,
            train_set,
            valid_sets=[valid_set],
            num_boost_round=5000,
            callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)],
     
        )

        y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
        rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
        rmse_scores.append(rmse)

    return np.mean(rmse_scores)  

# ================= 2. RUN OPTUNA STUDY =================
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

print("Best RMSE:", study.best_value)
print("Best hyperparameters:", study.best_params)

# ================= 3. TRAIN FINAL MODEL DENGAN PARAM TERBAIK =================
best_params = study.best_params
best_params.update({
    "objective": "regression",
    "metric": "rmse",
    "verbosity": -1
})

train_set = lgb.Dataset(X, y)
final_model = lgb.train(
    best_params,
    train_set,
    num_boost_round=5000  
)


# **7. Prediction Function for the Test Set**

* The `predict()` function is written as required by the competition API.
* Steps inside the function:

  1. Select the same feature columns.
  2. Apply the saved scaler.
  3. Use the trained LightGBM model to predict `forward_returns`.
  4. Convert those continuous returns into **discrete trading signals (0, 1, 2)** using quantiles.

Signal rules:

| Signal | Meaning                                |
| ------ | -------------------------------------- |
| 0      | Return ≤ 0                             |
| 1      | Return > 0 and ≤ 75th percentile (q75) |
| 2      | Return > q75                           |

---


In [ ]:
def convert_ret_to_signal(ret_arr: np.ndarray) -> np.ndarray:
    """
    Convert an array of returns to discrete trading signals 0, 1, 2
    """
    q75 = max(0, np.quantile(ret_arr, 0.75))

    signal = np.zeros_like(ret_arr, dtype=int)

    signal[(ret_arr > 0) & (ret_arr <= q75)] = 1

    signal[ret_arr > q75] = 2

    return signal


In [ ]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

features = list(X.columns)

def predict(test: pl.DataFrame) -> np.ndarray:
    df_test = test.select(features)

    X_test_np = df_test.to_numpy()

    scaler = joblib.load("scaler.pkl")  
    X_test_scaled = scaler.transform(X_test_np)  
    y_pred = final_model.predict(
        X_test_scaled, 
        num_iteration=final_model.best_iteration
    )
    

    signals = convert_ret_to_signal(y_pred)

    return float(signals)

In [ ]:
import kaggle_evaluation.default_inference_server
import os

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))


### **Summary**

> A minimal, functioning pipeline using LightGBM + Optuna for the Hull Tactical Market Prediction data.
> It serves purely as a **foundation** to verify that preprocessing, tuning, and prediction work correctly before deeper modeling begins.
>
> If this baseline gives you a head start, an **upvote** would be appreciated 🙏
